In [ ]:

from collections import Counter
import re
import string
from random import shuffle

from subprocess import call
from IPython.display import Image

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import spacy
from scipy import stats

from textblob import TextBlob

import textstat

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

from scipy import stats

import spacy

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from yellowbrick.classifier import ClassificationReport

from empath import Empath

from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_graphviz

In [ ]:
Regular_Comments=pd.read_csv("regular_comments.csv", error_bad_lines=False)

In [ ]:
Russians_Bots=pd.read_csv("russian_comments.csv")

In [ ]:
Regular_Comments=Regular_Comments[["body",'controversiality','created_utc',"subreddit","score"]]

In [ ]:
Russian_Bots=Russians_Bots[["body","controversiality","created_utc","subreddit.display_name_prefixed","score"]]

In [ ]:
Regular_Comments.head()

# Analiziranje subreddita

In [ ]:
Regular_Comments.describe()

In [ ]:
#Najpopularniji subredditi medju botovima
Russian_Bots.head()
ax=Russian_Bots["subreddit.display_name_prefixed"].value_counts()[:10].plot(kind="bar",title="Najpopularniji subredditi medju botovima")
ax.set_xlabel("Subreddit")
ax.set_ylabel("Broj komentara")

In [ ]:
#Najpopularniji subredditi medju regularnim komentarima
Regular_Comments.head()
ax=Regular_Comments["subreddit"].value_counts()[:10].plot(kind="bar",title="Najpopularniji subredditi medju regularnim korisnicima")
ax.set_xlabel("Ime subreddita")
ax.set_ylabel("Broj komentara sa subreddita")

In [ ]:
#Broj regularnih komentara
len(Regular_Comments.drop_duplicates())

In [ ]:
#Semplovanje regularnih komentara da ih bude isto koliko i ruskih bot komentara
Regular_Comments_Sample=Regular_Comments.sample(n=len(Russian_Bots),random_state=1)

In [ ]:
#Najpopularniji subredditi medju semplovanim komentarima
Regular_Comments_Sample["subreddit"].value_counts()[:10].plot(kind="bar")

In [ ]:
#Koliko najpopularnijih subreddita ima na oba dataseta
Regular_Comments_Sample["subreddit"].value_counts()[:20].index.isin(Regular_Comments["subreddit"].value_counts()[:20].index)

In [ ]:
#Dvadeset najcescih subreddita
Regular_Comments["subreddit"].value_counts()[:20].index

# Analiziranje upvote-ova

In [ ]:
#Najcesci broj upvote-ova na komentarima ruskih botova
print(Russian_Bots.score.value_counts()[:10])

In [ ]:
#Najcesci broj upvote-ova na komentarima regularnih korisnika
print(Regular_Comments_Sample.score.value_counts()[:10])

In [ ]:
#Mean od upvote-ova komentara
print("Mean upvote-ova regularnih {}".format
     (np.mean(Regular_Comments_Sample.score)))

print("Mean botova  {}".format
     ((np.mean(Russian_Bots.score))))


In [ ]:
#100 percentil upvote-ova regularnih 
print(np.percentile(Regular_Comments_Sample.score.dropna(),100))

In [ ]:
#isto za trolove
print(np.percentile(Russian_Bots.score.dropna(),100))

In [ ]:
#Pravljenje binova po upvote-ovima za regularne
bins=[-10,0,10,100,500,1000]
digitized=np.digitize(Regular_Comments_Sample.score, bins)
bin_means=[Regular_Comments_Sample.score[digitized == i].mean() for i in range(0, len(bins))]
print(bin_means) #mean upvote-ova za svaki bin

In [ ]:
#Vizualizacija distribucije za binove
ax=sns.countplot(digitized)
ax.set(xlabel='bin', ylabel='Broj komentara',title="Distribucija upvote-ova regularnih")

In [ ]:
#Binovi za botove
bins = [-10,0,10,100,500,1000]
digitized = np.digitize(Russian_Bots.score, bins)
bin_means = [Russian_Bots.score[digitized == i].mean() for i in range(0, len(bins))]

In [ ]:
ax=sns.countplot(digitized)
ax.set(xlabel='bin', ylabel='Broj komentara',title="Distribucija upvote-ova botova")

# Analiziranje vremena postovanja

In [ ]:
#utc u datetime
Russian_Bots["date"] = pd.to_datetime(Russian_Bots['created_utc'],unit='s')
Regular_Comments_Sample["date"] = pd.to_datetime(Regular_Comments_Sample['created_utc'],unit='s')
#warning je false-positive, ima objasnjenje na stacku zasto, nisam istrazio do kraja jer radi sve

In [ ]:
#utc u datetime
Russian_Bots["Time"]=Russian_Bots["date"].dt.hour.dropna()
Regular_Comments_Sample["Time"]=Regular_Comments_Sample["date"].dt.hour.dropna()


In [ ]:
ax=sns.countplot(Regular_Comments_Sample.Time)
ax.set(title="Distribucija vremena postovanja komentara",xlabel="vreme",ylabel="broj komentara")

In [ ]:
ax=sns.countplot(Russian_Bots.Time)
ax.set(title="Distribucija vremena postovanja komentara za botove",xlabel="vreme",ylabel="broj komentara")

In [ ]:
#Da li je distribucija vremena postovanja statisticki znacajna
stats.ks_2samp(Russian_Bots.Time,  Regular_Comments_Sample.Time)

In [ ]:
#Datumi sa najvise komentara botova
Russian_Bots["date"].dt.date.value_counts()[:50]

In [ ]:
#Datumi sa najvise komentara regularnih korisnika 
Regular_Comments_Sample["date"].dt.date.value_counts()[:20]

"Named Entities"

In [ ]:
nlp = spacy.load('en_core_web_sm')
#u novom spacy mora ovako, ranije moglo samo en, zna da baguje sa starijom verzijom spacy-ja

In [ ]:

#sredjivanje datasetova od praznina, spaceova...
Russian_Bots.body=Russian_Bots.body.str.replace("(\r)|(\n)","")
Regular_Comments_Sample.body=Regular_Comments_Sample.body.str.replace("(\r)|(\n)","")

In [ ]:
def named_entities(x):
    """dobavljanje named entities"""
    doc=nlp(x)
    named_=[]
    for doc in doc.ents:
        if doc == None:
            return named_
        else:
            named_.append(doc)
    return named_

In [ ]:
#Named entities za regularne
Regular_Comments_Sample["Named_entities"]=Regular_Comments_Sample.body.apply(func=lambda x: named_entities(x))

In [ ]:
#NE za botove
Russian_Bots["Named_entities"]=Russian_Bots.body.apply(func=lambda x: named_entities(x))

In [ ]:
def get_polarity(x):
  """Polarnost svakog komentara."""
  b = TextBlob(x)
  return b.sentiment.polarity

In [ ]:
#Polarnost regularnih
Regular_Comments_Sample["Polarity"]=Regular_Comments_Sample.body.apply(func=lambda x: get_polarity(x))
np.mean(Regular_Comments_Sample["Polarity"])

In [ ]:
#Polarnost botova
Russian_Bots["Polarity"]=Russian_Bots.body.apply(func=lambda x: get_polarity(x))
np.mean(Russian_Bots["Polarity"])

Flesch-Kinkaid

In [ ]:
#Flesch Kincaid za botove
Russian_Bots["Flesch_Kincaid"]=Russian_Bots.body.apply(func=lambda x: textstat.flesch_kincaid_grade(x))
np.mean(Russian_Bots["Flesch_Kincaid"])

In [ ]:
#Isto za regularne
Regular_Comments_Sample["Flesch_Kincaid"]=Regular_Comments_Sample.body.apply(func=lambda x: textstat.flesch_kincaid_grade(x))
np.mean(Regular_Comments_Sample["Flesch_Kincaid"])

# Analiziranje broja kontroverznih komentara

In [ ]:
#Broj kontroverznih bot komentara
Russian_Bots.controversiality.value_counts()

In [ ]:
#Broj kontroverznih regularnih
Regular_Comments_Sample.controversiality.value_counts()

# Leksika

In [ ]:
lexicon = Empath()

In [ ]:
#5 leksickih kategorija najcesce vezanih za ruske komentare, izvor ideje je naucni rad : 
lexicon.create_category("russia",["russia","putin","moscow","kremlin","KGB","ukraine","russian"],model="reddit")
lexicon.create_category("social_media",["fake_news","twitter_bot","social_media"],model="reddit")
lexicon.create_category("trump",["donald_trump","POTUS","michael_flynn","russian_collusion","mueller_investigation","putin"],model="reddit")
lexicon.create_category("FBI",["FBI","investigation","robert","comey","flynn"],model="reddit")
lexicon.create_category("hilary",["DNC","emails","bernie"],model="reddit")

In [ ]:
def Lexical_Score(x):
    """Dodeljivanje leksickog score-a svakom komentaru u odnosu na kontekst recenice"""
    c=0
    analyze=lexicon.analyze(x,categories=['russia,social_media',"trump","FBI","hilary"],normalize=True)
    attrs=[w for k,w in analyze.items() if w >0]
    if len(attrs)>0:
        for attr in attrs:
            c+=attr
    return c

In [ ]:
Russian_Bots["Lexical_Score"]=Russian_Bots.body.apply(func=lambda x:Lexical_Score(x))

In [ ]:
Regular_Comments_Sample["Lexical_Score"]=Regular_Comments_Sample.body.apply(func=lambda x:Lexical_Score(x))

# Dodavanje jos nekih osnovnih feature vezanih za broj reci odnosno recenice

In [ ]:
#broj reci
Russian_Bots['word_count']=Russian_Bots['body'].apply(lambda x: len(x.split()))
Regular_Comments_Sample['word_count']=Regular_Comments_Sample['body'].apply(lambda x: len(x.split()))

In [ ]:
#Znakovi interpunkcije
Russian_Bots['punctuation_count']=Russian_Bots['body'].apply(lambda x: len("".join(punc for punc in x if punc in string.punctuation))) 
Regular_Comments_Sample['punctuation_count']=Regular_Comments_Sample['body'].apply(lambda x: len("".join(punc for punc in x if punc in string.punctuation))) 

In [ ]:
#Pocetna velika slova
Russian_Bots['upper_case_word_count'] = Russian_Bots.body.apply(lambda x: len([word for word in x.split() if word.isupper()]))
Regular_Comments_Sample['upper_case_word_count'] = Regular_Comments_Sample.body.apply(lambda x: len([word for word in x.split() if word.isupper()]))

In [ ]:
#Izdvajanje bitnih kolona
Regular_Comments_Sample=Regular_Comments_Sample[["Time","Named_entities","Polarity",
                     "Flesch_Kincaid","Lexical_Score","word_count","punctuation_count",
                     "upper_case_word_count"]]
Russian_Bots=Russian_Bots[["Time","Named_entities","Polarity",
                     "Flesch_Kincaid","Lexical_Score","word_count","punctuation_count",
                     "upper_case_word_count"]]


In [ ]:
#Dodavanje labela
Regular_Comments_Sample["Label"]=["0" for x in range(len(Regular_Comments_Sample))]
Russian_Bots["Label"]=["1" for x in range(len(Regular_Comments_Sample))]

In [ ]:
#Spajanja datasetova
total=Regular_Comments_Sample.append(Russian_Bots)

In [ ]:
#Ukupan broj named entities za svaki komentar
total["Named_entities"]=total.Named_entities.apply(lambda x:len(x))

#Baseline test

In [ ]:
#oznacavanje bot i regularnih komentara
Russian_Bots["Troll_Status"]=["Russian" for x in range(len(Russian_Bots))]
Regular_Comments_Sample["Troll_Status"]=["Non-Russian" for x in range(len(Russian_Bots))]
appended=np.array(Regular_Comments_Sample.append(Russian_Bots)["Troll_Status"])

In [ ]:
#Lista sa random oznakama da li je komentar bot ili regularan
actual=list(pd.Series(appended))
troll=["Russian" for x in range(len(Russian_Bots))]
not_troll=["Non-Russian" for x in range(len(Russian_Bots))]
combined=troll+not_troll

In [ ]:

shuffle(combined)
shuffle(appended)

In [ ]:
#Accuracy, logicno je da je oko 50% posto je random
x=0
for label in range(len(actual)):
    if actual[label]==combined[label]:
        x+=1
print(x/len(combined)*100)

In [ ]:
#Random predikcije proveravanje rezultata
actual_converted=[]
test=[]
for label in actual:
  if label=="Non-Russian":
    actual_converted.append(0)
  else:
    actual_converted.append(1)
for label in combined:
  if label=="Non-Russian":
    test.append(0)

  else:
    test.append(1)

In [ ]:
# Recall
metrics.recall_score(actual_converted,test)

In [ ]:
# AUC
metrics.roc_auc_score(actual_converted,test)

# Random Forest

In [ ]:
#novi index
total.index=list(range(len(total)))


In [ ]:
#Random mesanje redova
shuffled_rows=total.reindex(np.random.permutation(total.index))

In [ ]:
#Izbacivanje N/A redova
shuffled_rows.dropna(inplace=True)

In [ ]:

X=shuffled_rows.iloc[:,:-1]#nezavisne promenljive
y=shuffled_rows.iloc[:,-1]#vrednosti koje se prediktuju

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
corr = X.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Korelacija izmedju parametara modela', fontsize=14)

In [ ]:
#Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [ ]:
#Standardizovanje podataka
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

In [ ]:
#Random Forest algoritam za klasifikovanje
Classifier = RandomForestClassifier(n_estimators=20, random_state=0)  
Classifier.fit(X_train, y_train)  
y_pred = Classifier.predict(X_test) 

In [ ]:
y_pred=y_pred.astype(int)
y_test_int=y_test.astype(int)

In [ ]:
# Recall
metrics.recall_score(y_test_int,y_pred)

In [ ]:
#auc
metrics.roc_auc_score(y_test_int,y_pred)

In [ ]:
visualizer = ClassificationReport(RandomForestClassifier() ,cmap='BuPu',title="Normal Random Forest Parameters",classes=["Non-Troll","Troll"], support=True)
visualizer.fit(X_train, y_train)  # Fitovanje vizualizatora i modela
visualizer.score(X_test, y_test)  # Evaluacija modela na test podacima
g = visualizer.poof()             # vizualizovanje podataka